In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Đọc dữ liệu từ các tệp đã cung cấp
btc_data = pd.read_csv('BTC.csv')  
gold_data = pd.read_csv('GOLD.csv')
vni_data = pd.read_csv('VNI.csv')

# Chuyển đổi cột ngày về dạng datetime
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
vni_data['Date'] = pd.to_datetime(vni_data['Date'])

btc_data['Cchange'] = pd.to_numeric(btc_data['Cchange'].str.rstrip('%').replace('#DIV/0!', np.nan), errors='coerce') / 100
vni_data['Cchange'] = pd.to_numeric(vni_data['Cchange'].str.rstrip('%').replace('#DIV/0!', np.nan), errors='coerce') / 100


gold_data = gold_data[(gold_data['Date'] >= '2021-01-01') & (gold_data['Date'] <= '2024-09-01')]
btc_data = btc_data[(btc_data['Date'] >= '2021-01-01') & (btc_data['Date'] <= '2024-09-01')]
vni_data = vni_data[(vni_data['Date'] >= '2021-01-01') & (vni_data['Date'] <= '2024-09-01')]
# Tạo figure và subplots
fig, axs = plt.subplots(3, 1, figsize=(12, 10))

# Vẽ biểu đồ cho Bitcoin (BTC)
axs[0].plot(btc_data['Date'], btc_data["Cchange"], color='gray')
axs[0].set_title('BTC')
axs[0].set_ylabel('Daily Return')

# Vẽ biểu đồ cho Gold
axs[1].plot(gold_data['Date'], gold_data["Vola"], color='gray')
axs[1].set_title('Vàng')
axs[1].set_ylabel('Daily Return')

# Vẽ biểu đồ cho VNI
axs[2].plot(vni_data['Date'], vni_data["Cchange"], color='gray')
axs[2].set_title('VNI')
axs[2].set_ylabel('Daily Return')

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'BTC.csv'

In [ ]:
btc_data_filtered = btc_data.loc[(btc_data['Date'] >= '2021-01-01') & (btc_data['Date'] <= '2024-09-01')][::-1]
vni_data_filtered = vni_data.loc[(vni_data['Date'] >= '2021-01-01') & (vni_data['Date'] <= '2024-09-01')]
gold_data_filtered = gold_data.loc[(gold_data['Date'] >= '2021-01-01') & (gold_data['Date'] <= '2024-09-01')]

# Chuyển đổi cột "Cchange" về kiểu số, bỏ đi ký tự %
btc_data_filtered['Cchange'] = pd.to_numeric(btc_data_filtered['Cchange'], errors='coerce')
vni_data_filtered['Cchange'] = pd.to_numeric(vni_data_filtered['Cchange'], errors='coerce')
gold_data_filtered['Vola'] = pd.to_numeric(gold_data_filtered['Vola'], errors='coerce')
# Loại bỏ các giá trị NaN
btc_returns = btc_data_filtered['Cchange'].dropna()
vnireturns = vni_data_filtered['Cchange'].dropna()
gold_returns = gold_data_filtered['Vola'].dropna()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, spearmanr

# Đổi tên Series
btc_new = btc_returns.rename("BTC")
gold_new = gold_returns.rename("Gold")
vninew = vnireturns.rename("VNI")

#reset index cho từng dữ liệu
btc_new = btc_new.reset_index(drop=True)
vninew= vninew.reset_index(drop=True)
gold_new= gold_new.reset_index(drop=True)

# Kết hợp dữ liệu (merge)
merged_data = pd.concat([btc_new, gold_new, vninew], axis=1)

# Tính toán ma trận tương quan Kendall và Spearman
kendall_corr = merged_data.corr(method='kendall')
spearman_corr = merged_data.corr(method='spearman')

# Vẽ heatmap cho Kendall
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.heatmap(kendall_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Kendall Rank Correlation')

# Vẽ heatmap cho Spearman
plt.subplot(1, 2, 2)
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Spearman Rank Correlation')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, jarque_bera

data = merged_data
# Tạo một DataFrame để lưu kết quả
results = pd.DataFrame(columns=['Mean', 'S.D.', 'Skewness', 'Kurtosis', 'Jarque-Bera', 'Jarque-Bera p-value'])

# Tính các chỉ số thống kê cho mỗi tài sản
for asset in ['Gold', 'VNI', 'BTC']:  
    mean = data[asset].mean()
    std_dev = data[asset].std()
    skewness = skew(data[asset])
    kurt = kurtosis(data[asset])
    jb_stat, jb_pvalue = jarque_bera(data[asset])
    
    results.loc[asset] = [mean, std_dev, skewness, kurt, jb_stat, jb_pvalue]

print("Descriptive Statistics for Gold, VNI, and BTC:")
print(results)

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch

def ljung_box_test(series, lags=20):
    result = acorr_ljungbox(series, lags=[lags], return_df=True)
    return {
        "Ljung-Box Q Statistic": result.iloc[0, 0],
        "p-value": result.iloc[0, 1],
        "No Autocorrelation": result.iloc[0, 1] > 0.05  
    }

def arch_lm_test(series):
    result = het_arch(series)
    return {
        "ARCH-LM Statistic": result[0],
        "p-value": result[1],
        "No ARCH Effect": result[1] > 0.05  
    }


ljung_box_results = []
arch_lm_results = []

for column in merged_data.columns:
    series = merged_data[column].dropna()
    ljung_box_results.append({"Column": column, **ljung_box_test(series)})
    arch_lm_results.append({"Column": column, **arch_lm_test(series)})


ljung_box_results_df = pd.DataFrame(ljung_box_results)
arch_lm_results_df = pd.DataFrame(arch_lm_results)

print(ljung_box_results_df)
print(arch_lm_results_df)

In [ ]:
from statsmodels.tsa.stattools import kpss, adfuller

data=merged_data

def kpss_test(series, column_name):
    result = kpss(series, regression='c', nlags="auto")
    output = {
        "Column": column_name,
        "KPSS Statistic": result[0],
        "p-value": result[1],
        "Critical Values": result[3],
        "Stationary": result[1] > 0.05  
    }
    return output


kpss_results = []
for column in data.columns:
    kpss_results.append(kpss_test(data[column].dropna(), column))

kpss_results_df = pd.DataFrame(kpss_results)

def adf_test(series, column_name):
    result = adfuller(series, autolag='AIC')
    output = {
        "Column": column_name,
        "ADF Statistic": result[0],
        "p-value": result[1],
        "Critical Values": result[4],
        "Stationary": result[1] < 0.05  
    }
    return output

results = []
for column in data.columns:
    results.append(adf_test(data[column].dropna(), column))

adf_results = pd.DataFrame(results)

print(kpss_results_df)
print(adf_results)

In [ ]:
import scipy.stats as stats

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Q-Q Plots for Cchange and Vola Returns')

# Q-Q plot for BTC 
stats.probplot(btc_returns, dist="norm", plot=axes[0])
axes[0].set_title('BTC Returns')

# Q-Q plot for GOLD 
stats.probplot(gold_returns, dist="norm", plot=axes[1])
axes[1].set_title('GOLD Returns')

# Q-Q plot for VNI 
stats.probplot(vnireturns, dist="norm", plot=axes[2])
axes[2].set_title('VNI Returns')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

def plot_acf_for_multiple_returns(returns_list, titles):
    fig, axes = plt.subplots(1, len(returns_list), figsize=(15, 5), sharey=True)
    for i, (returns, title) in enumerate(zip(returns_list, titles)):
        plot_acf(returns, lags=40, alpha=0.05, ax=axes[i])
        axes[i].set_title(f'ACF ({title}) Returns')
    plt.tight_layout()
    plt.show()

returns_list = [btc_returns, gold_returns, vnireturns]
titles = ['BTC', 'GOLD', 'VNI']

plot_acf_for_multiple_returns(returns_list, titles)

In [ ]:
import pandas as pd
from arch import arch_model
import matplotlib.pyplot as plt

def fit_gjr_garch_and_get_residuals(returns, title):

    model = arch_model(returns, vol='Garch', p=1, o=1, q=1, dist='Normal')
    fitted_model = model.fit(disp='off')
    
    # Lấy phần dư
    residuals = fitted_model.resid
    
    print(f"Summary for {title} GJR-GARCH Model:")
    print(fitted_model.summary())
    
    return residuals

btc_residuals = fit_gjr_garch_and_get_residuals(btc_returns, 'BTC')
gold_residuals = fit_gjr_garch_and_get_residuals(gold_returns, 'GOLD')
vni_residuals = fit_gjr_garch_and_get_residuals(vnireturns, 'VNI')

# Plot residuals
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(btc_residuals)
plt.title('Residuals for BTC Returns')

plt.subplot(1, 3, 2)
plt.plot(gold_residuals)
plt.title('Residuals for GOLD Returns')

plt.subplot(1, 3, 3)
plt.plot(vni_residuals)
plt.title('Residuals for VNI Returns')

plt.tight_layout()
plt.show()

import seaborn as sns
from scipy.stats import norm

sns.histplot(btc_residuals, kde=True, stat="density", label="BTC Residuals")
sns.histplot(gold_residuals, kde=True, stat="density", label="GOLD Residuals", color='orange')
sns.histplot(vni_residuals, kde=True, stat="density", label="VNI Residuals", color='green')
plt.legend()
plt.title('Distribution of Residuals')
plt.show()

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Chuẩn hóa phần dư
def normalize_residuals(residuals):
    scaler = StandardScaler()
    return scaler.fit_transform(residuals.values.reshape(-1, 1)).flatten()

btc_residuals_norm = normalize_residuals(btc_residuals)
gold_residuals_norm = normalize_residuals(gold_residuals)
vni_residuals_norm = normalize_residuals(vni_residuals)

residuals_df = pd.DataFrame({
    'BTC': btc_residuals_norm,
    'GOLD': gold_residuals_norm,
    'VNI': vni_residuals_norm
})
import seaborn as sns
import matplotlib.pyplot as plt

# Vẽ ma trận scatter plot
sns.pairplot(
    residuals_df, 
    diag_kind='kde',  
    plot_kws={'alpha': 0.6, 's': 15}  
)

plt.suptitle('Scatter Plot Matrix of GJR-GARCH Residuals', y=1.02)
plt.show()

In [ ]:
from scipy.stats import genpareto, norm, chi2
import numpy as np
from copulas.bivariate import Clayton, Frank

def compute_gng_and_uniform(residuals, title=""):
    
    lower_threshold = np.quantile(residuals, 0.05)  
    upper_threshold = np.quantile(residuals, 0.95)  

   
    left_tail = residuals[residuals < lower_threshold]
    body = residuals[(residuals >= lower_threshold) & (residuals <= upper_threshold)]
    right_tail = residuals[residuals > upper_threshold]
    

    left_params = genpareto.fit(-left_tail)  
    right_params = genpareto.fit(right_tail)


    def gng_cdf(x):
        if x < lower_threshold:
            return genpareto.cdf(-x, *left_params) * 0.05
        elif x > upper_threshold:
            return 0.95 + genpareto.cdf(x - upper_threshold, *right_params) * 0.05
        else:
            return 0.05 + norm.cdf(x, loc=body.mean(), scale=body.std()) * 0.9

    cdf_vectorized = np.vectorize(gng_cdf)
    uniform_data = cdf_vectorized(residuals)


    return uniform_data

# Tính toán theo phân phối GNG và uniform đối với phần dư của mô hình GARCH BTC, GOLD và VNI
btc_uniform_data = compute_gng_and_uniform(btc_residuals, title='BTC')
gold_uniform_data = compute_gng_and_uniform(gold_residuals, title='GOLD')
vni_uniform_data = compute_gng_and_uniform(vni_residuals, title='VNI')

def compute_log_likelihood_and_p_value(copula, data):
    try:
        # Tính log-likelihood của mô hình fitted
        pdf = copula.probability_density(data)
        log_likelihood = np.sum(np.log(pdf))

        # Tính log-likelihood của mô hình độc lập (independence copula)
        independent_log_likelihood = np.sum(np.log(data[:, 0]) + np.log(data[:, 1]))

        # Tính giá trị kiểm định Likelihood Ratio Test (LRT)
        likelihood_ratio_stat = 2 * (log_likelihood - independent_log_likelihood)
        p_value = 1 - chi2.cdf(likelihood_ratio_stat, df=1)  # df = 1 do chỉ có 1 tham số

        return log_likelihood, likelihood_ratio_stat, p_value
    except Exception as e:
        print(f"Error during computation: {e}")
        return "N/A", "N/A", "N/A"

urc_data = pd.DataFrame({
    'BTC': btc_uniform_data,
    'GOLD': gold_uniform_data,
    'VNI': vni_uniform_data
})

# Lấy tên các cột để phân tích từng cặp đối tượng
columns = urc_data.columns
num_columns = len(columns)

# Lưu trữ kết quả tính toán
results = []

# Lặp qua từng cặp cột để phân tích bằng các mô hình Copula
for i in range(num_columns):
    for j in range(i + 1, num_columns):
        col1 = columns[i]
        col2 = columns[j]
        
        # Chuẩn bị dữ liệu của cặp hiện tại
        data1 = urc_data[col1]
        data2 = urc_data[col2]
        copula_data_np = np.column_stack((data1, data2))

        # Phân tích bằng Frank Copula
        frank_copula = Frank()
        try:
            frank_copula.fit(copula_data_np)
            frank_theta = frank_copula.theta
            frank_log_likelihood, frank_lrt_stat, frank_p_value = compute_log_likelihood_and_p_value(frank_copula, copula_data_np)
            frank_result = f"Theta: {frank_theta:.4f}, Log-Lik: {frank_log_likelihood:.4f}, LRT: {frank_lrt_stat:.4f}, p: {frank_p_value:.4f}"
        except Exception as e:
            frank_result = f"Error: {e}"

        # Phân tích bằng Clayton Copula
        clayton_copula = Clayton()
        try:
            clayton_copula.fit(copula_data_np)
            clayton_theta = clayton_copula.theta
            clayton_log_likelihood, clayton_lrt_stat, clayton_p_value = compute_log_likelihood_and_p_value(clayton_copula, copula_data_np)
            clayton_result = f"Theta: {clayton_theta:.4f}, Log-Lik: {clayton_log_likelihood:.4f}, LRT: {clayton_lrt_stat:.4f}, p: {clayton_p_value:.4f}"
        except Exception as e:
            clayton_result = f"Error: {e}"

        # Lưu kết quả vào danh sách
        results.append({
            'Pair': f'{col1}-{col2}',
            'Frank': frank_result,
            'Clayton': clayton_result
        })

results_df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)  
print("Copula Model Estimation Results:")
print(results_df)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm, t, kendalltau

ranked_data = urc_data.apply(lambda x: rankdata(x) / (len(x) + 1), axis=0)

kendall_matrix = np.zeros((len(columns), len(columns)))
p_values_matrix = np.zeros((len(columns), len(columns)))

for i in range(len(columns)):
    for j in range(len(columns)):
        if i == j:
            kendall_matrix[i, j] = 1
            p_values_matrix[i, j] = 0  
        else:
            tau, p_value = kendalltau(ranked_data[columns[i]], ranked_data[columns[j]])
            kendall_matrix[i, j] = tau
            p_values_matrix[i, j] = p_value


gaussian_correlation_matrix = np.sin(np.pi / 2 * kendall_matrix)

df = 5 
student_correlation_matrix = np.zeros_like(gaussian_correlation_matrix)

for i in range(len(columns)):
    for j in range(len(columns)):
        if i == j:
            student_correlation_matrix[i, j] = 1
        else:
            student_correlation_matrix[i, j] = gaussian_correlation_matrix[i, j] * (df / (df - 2))

results_gaussian = pd.DataFrame(index=columns, columns=columns)
results_student = pd.DataFrame(index=columns, columns=columns)

for i in range(len(columns)):
    for j in range(len(columns)):
        if i == j:
            results_gaussian.iloc[i, j] = f"{gaussian_correlation_matrix[i, j]:.4f} [0]"
            results_student.iloc[i, j] = f"{student_correlation_matrix[i, j]:.4f} [0]"
        else:
            results_gaussian.iloc[i, j] = f"{gaussian_correlation_matrix[i, j]:.4f} [{p_values_matrix[i, j]:.3f}]"
            results_student.iloc[i, j] = f"{student_correlation_matrix[i, j]:.4f} [{p_values_matrix[i, j]:.3f}]"

print("Gaussian Copula Correlation Matrix with P-values:")
print(results_gaussian)

print("\nStudent-t Copula Correlation Matrix with P-values:")
print(results_student)

In [ ]:
import numpy as np
from scipy.stats import norm, t
import pandas as pd
from scipy.stats import multivariate_normal

# Input: Hệ số tương quan từ Copula và lợi nhuận tài sản
copula_results = {
    "BTC-Gold": {"Frank": -0.2820, "Clayton": -0.0607, "T-Student": -0.0819, "Gaussian": -0.0492},
    "BTC-VNI": {"Frank": 0.1241, "Clayton": 0.0279, "T-Student": 0.036, "Gaussian": 0.0216},
    "VNI-Gold": {"Frank": 0.1968, "Clayton": 0.0447, "T-Student": 0.0572, "Gaussian": 0.0343}
}


# Tổ chức lại dữ liệu
returns_data = merged_data

# Bước 1: Mô phỏng lợi nhuận danh mục đầu tư từ Copula
def simulate_portfolio_returns(copula_rho, asset1_returns, asset2_returns, weights, num_simulations=100000):
    cov_matrix = np.array([[1, copula_rho], [copula_rho, 1]])
    normal_samples = multivariate_normal.rvs(mean=[0, 0], cov=cov_matrix, size=num_simulations)
    uniform_samples = norm.cdf(normal_samples)
    asset1_simulated = np.quantile(asset1_returns, uniform_samples[:, 1])
    asset2_simulated = np.quantile(asset2_returns, uniform_samples[:, 1])
    portfolio_returns = weights[0] * asset1_simulated + weights[1] * asset2_simulated
    return portfolio_returns

# Bước 2: Tính toán VaR
def calculate_var(portfolio_returns, confidence_level=0.99):
    var = np.percentile(portfolio_returns, (1 - confidence_level) * 100)
    return var

# Bước 3: Historical Simulation (HS)
def historical_var(asset1_returns, asset2_returns, weights, confidence_level=0.99):
    portfolio_returns = weights[0] * asset1_returns + weights[1] * asset2_returns
    var = np.percentile(portfolio_returns, (1 - confidence_level) * 100)
    return var

# Bước 4: Variance-Covariance (VC)
def variance_covariance_var(asset1_returns, asset2_returns, weights, confidence_level=0.99):
    portfolio_mean = weights[0] * np.mean(asset1_returns) + weights[1] * np.mean(asset2_returns)
    portfolio_variance = (weights[0]**2 * np.var(asset1_returns) +
                          weights[1]**2 * np.var(asset2_returns) +
                          2 * weights[0] * weights[1] * np.cov(asset1_returns, asset2_returns)[0, 1])
    portfolio_std = np.sqrt(portfolio_variance)
    var = portfolio_mean - norm.ppf(confidence_level) * portfolio_std
    return var

# Tính toán VaR cho từng cặp tài sản và phương pháp
confidence_levels = [0.99, 0.95, 0.90]
weights = [0.5, 0.5]
results = []

for pair, copulas in copula_results.items():
    asset1, asset2 = pair.split("-")
    asset1_returns = returns_data[asset1].values
    asset2_returns = returns_data[asset2].values
    
    for copula, rho in copulas.items():
        simulated_returns = simulate_portfolio_returns(rho, asset1_returns, asset2_returns, weights)
        for cl in confidence_levels:
            results.append({
                "Pair": pair,
                "Method": copula,
                "Confidence Level": f"VaR({int(cl * 100)}%)",
                "VaR": calculate_var(simulated_returns, cl)
            })
    
    for cl in confidence_levels:
        results.append({
            "Pair": pair,
            "Method": "HS",
            "Confidence Level": f"VaR({int(cl * 100)}%)",
            "VaR": historical_var(asset1_returns, asset2_returns, weights, cl)
        })
        results.append({
            "Pair": pair,
            "Method": "VC",
            "Confidence Level": f"VaR({int(cl * 100)}%)",
            "VaR": variance_covariance_var(asset1_returns, asset2_returns, weights, cl)
        })


results_df = pd.DataFrame(results)

pivot_table = results_df.pivot_table(index=["Pair", "Confidence Level"], columns="Method", values="VaR")
pivot_table = pivot_table.reset_index()
print(pivot_table)


import pandas as pd
import numpy as np
from scipy.stats import chi2
from statsmodels.stats.diagnostic import acorr_ljungbox

# Sử dụng pivot_table từ phần tính toán VaR
backtesting_data = pivot_table  # Kết quả từ đoạn code trước

# Hàm kiểm định VaR
def var_backtest(returns_data, backtesting_data, confidence_level, expected_exceedances):
    exceedances = returns_data < -backtesting_data  # Kiểm tra vi phạm
    num_exceedances = exceedances.sum()  # Số lần vượt mức
    n = len(returns_data)  # Tổng số quan sát

    # Kiểm định Unconditional Coverage (UC)
    p_hat = num_exceedances / n  # Tỉ lệ vi phạm thực tế
    if p_hat == 0 or p_hat == 1:
        uc_stat = np.inf
        uc_p_value = 0.0
    else:
        uc_stat = -2 * (
            (n - num_exceedances) * np.log(1 - confidence_level) +
            num_exceedances * np.log(confidence_level) -
            (n - num_exceedances) * np.log(1 - p_hat) -
            num_exceedances * np.log(p_hat)
        )
        uc_p_value = 1 - chi2.cdf(uc_stat, 1)

    # Kiểm định Independence (IND)
    if num_exceedances > 0:
        ind_test_result = acorr_ljungbox(exceedances.astype(int), lags=[1], return_df=True)
        ind_stat = ind_test_result['lb_stat'].iloc[0]
        ind_p_value = ind_test_result['lb_pvalue'].iloc[0]
    else:
        ind_stat = np.nan
        ind_p_value = np.nan

    # Kiểm định Conditional Coverage (CC)
    if np.isfinite(uc_stat) and np.isfinite(ind_stat):
        cc_stat = uc_stat + ind_stat
        cc_p_value = 1 - chi2.cdf(cc_stat, 2)
    else:
        cc_stat = np.nan
        cc_p_value = np.nan

    # Kết quả kiểm định
    test_result = "AAA" if uc_p_value > 0.05 and ind_p_value > 0.05 and cc_p_value > 0.05 else \
              "RAA" if uc_p_value <= 0.05 and ind_p_value > 0.05 and cc_p_value > 0.05 else \
              "ARA" if uc_p_value > 0.05 and ind_p_value <= 0.05 and cc_p_value > 0.05 else \
              "RRA" if uc_p_value <= 0.05 and ind_p_value <= 0.05 and cc_p_value > 0.05 else \
              "RAR" if uc_p_value <= 0.05 and ind_p_value > 0.05 and cc_p_value <= 0.05 else \
              "ARR" if uc_p_value > 0.05 and ind_p_value <= 0.05 and cc_p_value <= 0.05 else "RRR"


    return {
        'num_exceedances': num_exceedances,
        'Expected exceedances': expected_exceedances,
        'UC Stat': uc_stat,
        'UC p-value': uc_p_value,
        'IND Stat': ind_stat,
        'IND p-value': ind_p_value,
        'CC Stat': cc_stat,
        'CC p-value': cc_p_value,
        'Test Result': test_result
    }


# Chuẩn bị kiểm định cho từng cặp tài sản, phương pháp, và mức tin cậy
confidence_levels = [0.99, 0.95, 0.90]
expected_exceedances = {0.99: 1000, 0.95: 5000, 0.90: 10000}  # Số lần vi phạm kỳ vọng dựa trên kích thước mẫu
backtest_results = []

for index, row in pivot_table.iterrows():
    pair = row["Pair"]
    confidence_level = float(row["Confidence Level"].replace("VaR(", "").replace("%)", "")) / 100
    for method in ["HS", "VC", "Gaussian", "T-Student", "Clayton", "Frank"]:
        # Lấy lợi nhuận thực tế từ returns_data
        asset1, asset2 = pair.split("-")
        returns = 0.5 * returns_data[asset1] + 0.5 * returns_data[asset2]  #tính lợi nhuận danh mục

        # Lấy giá trị VaR từ bảng pivot_table
        var_value = row[method]

        # Kiểm định VaR
        test_results = var_backtest(returns, var_value, confidence_level, expected_exceedances[confidence_level])
        backtest_results.append({
            "Pair": pair,
            "Method": method,
            "Confidence Level": row["Confidence Level"],
            "Exceedances": test_results['num_exceedances'],
            "Expected Exceedances": test_results['Expected exceedances'],
            "UC Stat": test_results['UC Stat'],
            "UC p-value": test_results['UC p-value'],
            "IND Stat": test_results['IND Stat'],
            "IND p-value": test_results['IND p-value'],
            "CC Stat": test_results['CC Stat'],
            "CC p-value": test_results['CC p-value'],
            "Test Result": test_results['Test Result']
        })

backtest_results_df = pd.DataFrame(backtest_results)

print("VaR Backtesting Results:")
print(backtest_results_df)

In [ ]:
from statsmodels.tsa.api import VAR
def calcAvgSpilloversTable(merged_data, forecast_horizon=10, lag_order=None):

	forecast_horizon = 10 if forecast_horizon is None else forecast_horizon
	
	model = VAR(merged_data)
	if lag_order is None:
		results = model.fit(ic='aic')
		lag_order = results.k_ar
	else:
		results = model.fit(lag_order)
	
	sigma_u = np.asarray(results.sigma_u)
	sd_u = np.sqrt(np.diag(sigma_u))
	
	fevd = results.fevd(forecast_horizon, sigma_u/sd_u)
	fe = fevd.decomp[:, -1, :]
	fevd = (fe / fe.sum(1)[:, None] * 100)

	cont_incl = fevd.sum(0)
	cont_to = fevd.sum(0) - np.diag(fevd)
	cont_from  = fevd.sum(1) - np.diag(fevd)
	spillover_index = 100 * cont_to.sum() / cont_incl.sum()

	names = model.endog_names
	spilloversTable = pd.DataFrame(fevd, columns=names).set_index([names])
	spilloversTable.loc['Cont_To'] = cont_to
	spilloversTable.loc['Cont_Incl'] = cont_incl
	spilloversTable = pd.concat([spilloversTable, pd.DataFrame(cont_from, columns=['Cont_From']).set_index([names])], axis=1)
	spilloversTable = pd.concat([spilloversTable, pd.DataFrame(cont_to - cont_from, columns=['Cont_Net']).set_index([names])], axis=1)
	spilloversTable.loc['Cont_To', 'Cont_From'] = cont_to.sum()
	spilloversTable.loc['Cont_Incl', 'Cont_From'] = cont_incl.sum()
	spilloversTable.loc['Cont_Incl', 'Cont_Net'] = spillover_index

	return spilloversTable, lag_order, forecast_horizon

spilloversTable, lag_order, forecast_horizon = calcAvgSpilloversTable(merged_data)
print(spilloversTable)